In [16]:
# !pip install "google-cloud-aiplatform>=1.38" vertexai
#TODO: gcloud auth application-default login to authenticate

## Preprocess input

In [17]:
predictions_root_dir = '/home/watson_chua/efs/axolotl/data/predictions/hansard/'
llama_predictions_path = predictions_root_dir + "llama3_8b_lora.jsonl"
gemma_predictions_path = predictions_root_dir + "gemma2_9b_lora.jsonl"
gpt4_predictions_path, out_subdir = "/home/watson_chua/efs/axolotl/data/input_data/data/gpt4_answers_by_hy_doc.jsonl", 'gpt4_normal'
# gpt4_predictions_path, out_subdir = "/home/watson_chua/efs/axolotl/data/input_data/data/gpt4_answers_by_hy_doc_concise.jsonl", 'gpt4_concise'


In [18]:
import json
with open(llama_predictions_path, 'r') as f:
    llama_lines = f.readlines()

with open(gemma_predictions_path, 'r') as f:
    gemma_lines = f.readlines()

with open(gpt4_predictions_path, 'r') as f:
    gpt4_lines = f.readlines()

llama_lines = [json.loads(l) for l in llama_lines]
gemma_lines = [json.loads(l) for l in gemma_lines]
gpt4_lines = [json.loads(l) for l in gpt4_lines]

In [19]:
len(llama_lines), len(gemma_lines), len(gpt4_lines)

(304, 304, 304)

In [20]:
llama_lines[0]

{'generation': 'As at end-January 2024, there were 101,464 persons who did not vote in the last Presidential election but have since applied to restore their names to the Registers of Electors. All such applications have been approved after being verified by the Elections Department.',
 'input': "<|start_header_id|>system<|end_header_id|>You are a public servant. Your task is to reply to a parliamentary question given a list of supporting points.<|eot_id|><|start_header_id|>user<|end_header_id|>\nQuestion:Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.\n\nSupporting points: **Title: Applications to Restore Names to Registers of Electors**\n\n**Executive Summary:**\n\nAs of the end of January 2024, a total of 101,464 non-voters from the 2023 Presidential Election have successfully applied to res

In [21]:
gpt4_lines[0]['question']

'Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.'

In [22]:
llama_lines[0]['input'].split("<|end_header_id|>\nQuestion:", maxsplit=1)[-1].split("\n\nSupporting points:")[0]

'Mr Leong Mun Wai asked the Prime Minister (a) since the conclusion of the 2023 Presidential Election, how many non-voters have applied to restore their names to the Registers of Electors; and (b) how many of these applications are successful.'

In [23]:
llama_questions = [q['input'].split("<|end_header_id|>\nQuestion:", maxsplit=1)[-1].split("\n\nSupporting points:")[0] for q in llama_lines]

In [24]:
for i,l in enumerate(gpt4_lines):
    if l['question'] not in llama_questions:
        print(i, l['question'])

In [25]:
import pandas as pd 
df_gpt4 = pd.DataFrame(gpt4_lines)

In [26]:
df_duplicated = df_gpt4[df_gpt4['question'].duplicated()]

In [27]:
df_gpt4_deduplicated = df_gpt4.drop_duplicates('question')

In [28]:
df_gpt4_deduplicated.head()

,status,reference_answer,summary_points,points,title,subtitle,question_speaker,answer_speaker,question,answer,filename,hypothetical_document,date,gpt4_answer_by_hy_doc
0,answered,,"[101,464 non-voters from the 2023 Presidential...","101,464 non-voters from the 2023 Presidential ...",Applications to Restore Names to Registers of ...,None,Mr Leong Mun Wai,Mr Chan Chun Sing (for the Prime Minister),Mr Leong Mun Wai asked the Prime Minister (a) ...,Mr Chan Chun Sing (for the Prime Minister): As...,sitting_2024-02-29,**Title: Applications to Restore Names to Regi...,2024-02-29 00:00:00,"As of the end of January 2024, a total of 101,..."
1,answered,,"[From 2021 to 2023, over 23,300 companies obta...","From 2021 to 2023, over 23,300 companies obtai...",Applications under Enterprise Financing Scheme,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: From 2021 to 2023, over 23,30...",sitting_2024-02-29,**Title: Applications under Enterprise Financi...,2024-02-29 00:00:00,"From 2021 to 2023, over 23,300 companies have ..."
2,answered,,"[About 36,400 companies have received support ...","About 36,400 companies have received support u...",Applications for Productivity Solutions Grant ...,None,Mr Liang Eng Hwa,Mr Gan Kim Yong,Mr Liang Eng Hwa asked the Minister for Trade ...,"Mr Gan Kim Yong: In the last three years, abou...",sitting_2024-02-29,### Applications for Productivity Solutions Gr...,2024-02-29 00:00:00,"Over the last three years, approximately 36,40..."
3,answered,,[Schools have autonomy to decide the end times...,Schools have autonomy to decide the end times ...,Time Guidelines for Schools to Cease Co-curric...,None,Mr Leong Mun Wai,Mr Chan Chun Sing,Mr Leong Mun Wai asked the Minister for Educat...,Mr Chan Chun Sing: Schools have the autonomy t...,sitting_2024-02-29,**Title: Time Guidelines for Schools to Cease ...,2024-02-29 00:00:00,"Yes, there are overarching guidelines for scho..."
4,answered,,[Rail operators have improved track design and...,Rail operators have improved track design and ...,Mitigation Measures to Reduce Noise from Railw...,None,Mr Leong Mun Wai,Mr Chee Hong Tat,Mr Leong Mun Wai asked the Minister for Transp...,"Over the years, rail operators have improved t...",sitting_2024-02-29,**Report: Mitigation Measures to Reduce Noise ...,2024-02-29 00:00:00,"**Reply:**\n\nMr Speaker,\n\nTo address the no..."


In [29]:
df_all_predictions = df_gpt4_deduplicated.copy()

In [30]:
df_all_predictions['llama3_answer'] = [l['generation']  for l in llama_lines]
df_all_predictions['gemma2_answer'] = [l['generation']  for l in gemma_lines]

In [31]:
output_root_dir = '/home/watson_chua/efs/axolotl/data/evaluations/hansard/'

In [33]:
df_all_predictions.to_csv(output_root_dir + out_subdir + '/consolidated_predictions.csv', index=False)

## Post Processing

In [40]:
import re
def replace_inner_quotes(text):
    split_result  = re.split("\"\s*reason\s*\"", text, maxsplit=1)
    winner_str = split_result[0] 
    reason_str = split_result[1][split_result[1].index('"')+1: split_result[1].rfind('"')].replace('"','').replace("\\", "")
    return winner_str + "\"reason\": \"" + reason_str.strip() + '"}'

def clean_json_output(output):
    output = output.strip()
    if output.startswith("```json"):
        output = output[7:]
    if output.endswith("```"):
        output = output[:-3]
    cleaned_output = replace_inner_quotes(output.strip())
    # cleaned_output = output.strip()


    try:
        json_data = json.loads(cleaned_output, strict=False)
    except json.JSONDecodeError as e:
        print(f"JSON decoding error: {e}")
        print(cleaned_output)
        return cleaned_output

    def clean_json(data):
        if isinstance(data, dict):
            return {key: clean_json(value) for key, value in data.items()}
        elif isinstance(data, list):
            return [clean_json(item) for item in data]
        elif isinstance(data, str):
            return "" if data.lower() in ["unknown", "na", "null"] else data
        else:
            print(data)
            return data
    
    cleaned_json_data = clean_json(json_data)
    # cleaned_output = json.dumps(cleaned_json_data, ensure_ascii=False)

    return cleaned_json_data

In [42]:
#Test
# for er in three_way_evaluation_results:
#     clean_json_output(er['gemini_flash_evaluation'])